In [2]:
import numpy as np
import pandas as pd
import pathlib as Path
import os
import operator
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import mobilenet_v2 as mnet_v2

In [3]:
IMG_SIZE = (224, 244)
BATCH_SIZE = 32
EPOCHS = 50
channels = 3

In [4]:
def img_processing(path):
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], path))
    filepath = pd.Series(path, name='Path').astype(str)
    labels = pd.Series(labels, name='Label')
    
    df = pd.concat([filepath, labels], axis=1) 
    df = df.sample(frac=1).reset_index(drop=True)
    return df

In [5]:
train_dir = Path.Path('../input/the-simpsons-characters-dataset/simpsons_dataset')
filepaths = list(train_dir.glob(r'**/*.jpg'))

df = img_processing(filepaths)
df.head()

In [39]:
# to drop labels that contain less images that average 
# char_dict = {}
# for char in os.listdir(train_dir):
#     char_dict[char] = len(os.listdir(os.path.join(train_dir, char)))

# sorted_d = dict( sorted(char_dict.items(), key=operator.itemgetter(1),reverse=True))
# avg_size = sum(sorted_d.values())/len(sorted_d.values())
# labels_to_drop = [k for k, v in char_dict.items() if char_dict[k] < avg_size ]
# df = df[~df.Label.isin(labels_to_drop)].reset_index()
# df.head()

In [7]:
print(f'Number of pictures: {df.shape[0]}\n')
print(f'Number of different labels: {len(df.Label.unique())}\n')
print(f'Labels: {df.Label.unique()}')

In [8]:
df_uq = df.copy().drop_duplicates(subset=['Label']).reset_index()
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_uq.Path[i]))
    ax.set_title(df_uq.Label[i])
plt.show()

In [9]:
train_df, test_df = train_test_split(df, train_size=.8, shuffle=True, random_state=0)

In [10]:
train_gen = ImageDataGenerator(preprocessing_function=mnet_v2.preprocess_input, validation_split=.2)
test_gen = ImageDataGenerator(preprocessing_function=mnet_v2.preprocess_input)

In [11]:
train_imgs = train_gen.flow_from_dataframe(dataframe=train_df, 
                                           x_col='Path', y_col='Label',
                                           target_size=IMG_SIZE, 
                                           class_mode='categorical',
                                           batch_size = BATCH_SIZE,
                                           subset='training',
                                           rotation_range=30,
                                            zoom_range=0.15,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.15,
                                            horizontal_flip=True,
                                            fill_mode="nearest"
                                          )
val_imgs = train_gen.flow_from_dataframe(dataframe=train_df, 
                                           x_col='Path', y_col='Label',
                                           target_size=IMG_SIZE, 
                                           class_mode='categorical',
                                           batch_size = BATCH_SIZE,
                                           subset='validation',
                                           rotation_range=30,
                                            zoom_range=0.15,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.15,
                                            horizontal_flip=True,
                                            fill_mode="nearest"
                                          )
test_imgs = test_gen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Path',
    y_col='Label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [12]:
pretrained_model = mnet_v2.MobileNetV2(
    input_shape = (IMG_SIZE[0], IMG_SIZE[1], channels),
    include_top=False,
    weights='imagenet',
    pooling='avg')
pretrained_model.trainable = False

In [13]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(42, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
history = model.fit(
    train_imgs,
    validation_data=val_imgs,
    epochs=EPOCHS,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [23]:
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()

In [24]:
pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

In [25]:
pred = model.predict(test_imgs)
pred = np.argmax(pred,axis=1)

labels = (train_imgs.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

# Display the result
print(f'The first 5 predictions: {pred[:5]}')

In [26]:
from sklearn.metrics import accuracy_score
y_test = list(test_df.Label)
acc = accuracy_score(y_test,pred)
print(f'Accuracy on the test set: {acc * 100:.2f}%')

In [27]:
from sklearn.metrics import classification_report
class_report = classification_report(y_test, pred, zero_division=1)
print(class_report)

In [28]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cf_matrix = confusion_matrix(y_test, pred, normalize='true')
plt.figure(figsize = (20,15))
sns.heatmap(cf_matrix, annot=False, xticklabels = sorted(set(y_test)), yticklabels = sorted(set(y_test)))
plt.title('Normalized Confusion Matrix', fontsize = 23)
plt.show()

In [29]:
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(15, 12),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Path.iloc[i]))
    ax.set_title(f"True: {test_df.Label.iloc[i].split('_')[0]}\nPredicted: {pred[i].split('_')[0]}")
plt.tight_layout()
plt.show()